## State space averaging

Biblioteke koje su nam potrebne

In [1]:
import sympy as sp

Promjenljive stanja + ulazne promjenljive

In [2]:
i_Lm = sp.symbols("i_Lm")
v_C2 = sp.symbols("v_C2")
i_out = sp.symbols("i_out")
v_in = sp.symbols("v_in")

Parametri sistema

In [12]:
R_eq = sp.symbols("R_eq")
R_C2 = sp.symbols("R_C2")
L_m = sp.symbols("L_m")
C_2 = sp.symbols("C_2")
n = sp.symbols("n")
D_0 = sp.symbols("D_0")

Matrice stanja i matrice ulaza

In [13]:
A1 = sp.Matrix([[-R_eq / L_m, 0], [0, 0]])
A1

Matrix([
[-R_eq/L_m, 0],
[        0, 0]])

In [14]:
A2 = sp.Matrix([[-R_C2 / (n**2 * L_m), -1 / (n * L_m)], [1 / (n * C_2), 0]])
A2

Matrix([
[-R_C2/(L_m*n**2), -1/(L_m*n)],
[       1/(C_2*n),          0]])

In [15]:
B1 = sp.Matrix([[0, 1 / L_m], [-1 / C_2, 0]])
B1

Matrix([
[     0, 1/L_m],
[-1/C_2,     0]])

In [16]:
B2 = sp.Matrix([[R_C2 / (n * L_m), 0], [-1 / C_2, 0]])
B2

Matrix([
[R_C2/(L_m*n), 0],
[      -1/C_2, 0]])

In [17]:
C1 = sp.Matrix([[0, 1]])
C1

Matrix([[0, 1]])

In [18]:
C2 = sp.Matrix([[R_C2, 0]])
C2

Matrix([[R_C2, 0]])

In [19]:
D1 = D2 = sp.Matrix([[-R_C2, 0]])
D1

Matrix([[-R_C2, 0]])

In [20]:
A = sp.simplify(A1 * D_0 + A2 * (1 - D_0))
A

Matrix([
[(-D_0*R_eq*n**2 + R_C2*(D_0 - 1))/(L_m*n**2), (D_0 - 1)/(L_m*n)],
[                           (1 - D_0)/(C_2*n),                 0]])

In [21]:
B = sp.simplify(B1 * D_0 + B2 * (1 - D_0))
B

Matrix([
[R_C2*(1 - D_0)/(L_m*n), D_0/L_m],
[                -1/C_2,       0]])

In [22]:
C = sp.simplify(C1 * D_0 + C2 * (1 - D_0))
C

Matrix([[R_C2*(1 - D_0), D_0]])

In [23]:
D = sp.simplify(D1 * D_0 + D2 * (1 - D_0))
D

Matrix([[-R_C2, 0]])

Vrijednost promjenljivih u ustaljenom stanju

In [24]:
I_out = sp.symbols("I_out")
V_in = sp.symbols("V_in")

In [25]:
u_ss = sp.Matrix([[I_out], [V_in]])
u_ss

Matrix([
[I_out],
[ V_in]])

In [26]:
x_ss = -sp.simplify(A.inv() * B * u_ss)
x_ss

Matrix([
[                                                                              -I_out*n/(D_0 - 1)],
[-D_0*(-D_0*I_out*R_C2 + D_0*V_in*n + I_out*R_C2 + I_out*R_eq*n**2 - V_in*n)/(D_0**2 - 2*D_0 + 1)]])

In [27]:
I_Lm = x_ss[0]
V_C2 = x_ss[1]

In [28]:
sp.factor(V_C2)

-D_0*(-D_0*I_out*R_C2 + D_0*V_in*n + I_out*R_C2 + I_out*R_eq*n**2 - V_in*n)/(D_0 - 1)**2

In [33]:
I_in = sp.factor(D_0 * I_Lm)
sp.factor(I_in.evalf(subs={R_eq: 0, R_C2: 0}))

-1.0*D_0*I_out*n/(1.0*D_0 - 1.0)

In [32]:
sp.factor(V_C2.evalf(subs={R_eq: 0, R_C2: 0}))

-1.0*D_0*V_in*n*(D_0 - 1)/(1.0*D_0 - 1.0)**2

Super! Tačan model!

Matrice $E$ i $F$

In [34]:
E = sp.simplify((A1 - A2) * x_ss + (B1 - B2) * u_ss)
E

Matrix([
[(-D_0*V_in - I_out*R_eq*n + V_in)/(L_m*(D_0**2 - 2*D_0 + 1))],
[                                       I_out/(C_2*(D_0 - 1))]])

In [35]:
F = sp.simplify((C1 - C2) * x_ss + (D1 - D2) * u_ss)
F

Matrix([[(D_0**2*I_out*R_C2 - D_0**2*V_in*n + D_0*I_out*R_C2*n - D_0*I_out*R_C2 - D_0*I_out*R_eq*n**2 + D_0*V_in*n - I_out*R_C2*n)/(D_0**2 - 2*D_0 + 1)]])

Usrednjen model

In [36]:
new_B = sp.Matrix([[B, E]])
new_B

Matrix([
[R_C2*(1 - D_0)/(L_m*n), D_0/L_m, (-D_0*V_in - I_out*R_eq*n + V_in)/(L_m*(D_0**2 - 2*D_0 + 1))],
[                -1/C_2,       0,                                        I_out/(C_2*(D_0 - 1))]])

In [37]:
new_D = sp.Matrix([[D, F]])
new_D

Matrix([[-R_C2, 0, (D_0**2*I_out*R_C2 - D_0**2*V_in*n + D_0*I_out*R_C2*n - D_0*I_out*R_C2 - D_0*I_out*R_eq*n**2 + D_0*V_in*n - I_out*R_C2*n)/(D_0**2 - 2*D_0 + 1)]])

Procjena ripple-a

In [38]:
f_s = sp.symbols("f_s")
deltaX = sp.simplify(
    ((A1 * D_0 - A2 * (1 - D_0)) * x_ss + (B1 * D_0 - B2 * (1 - D_0)) * u_ss)
    / (4 * f_s)
)
deltaX

Matrix([
[D_0*(D_0*V_in + I_out*R_eq*n - V_in)/(2*L_m*f_s*(D_0 - 1))],
[                                    -D_0*I_out/(2*C_2*f_s)]])

In [39]:
delta_I_Lm = deltaX[0]
delta_I_Lm

D_0*(D_0*V_in + I_out*R_eq*n - V_in)/(2*L_m*f_s*(D_0 - 1))

In [40]:
delta_V_C2 = deltaX[1]
delta_V_C2

-D_0*I_out/(2*C_2*f_s)